In [1]:
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
chromePath ='C:\seleniumdriver\chromedriver_win32\chromedriver.exe'

In [3]:
# Lấy cookie
driver=webdriver.Chrome(chromePath)
url = 'https://mbasic.facebook.com/'
driver.get(url)
time.sleep(3)
email_box = driver.find_element(by=By.XPATH, value='//*[@id="m_login_email"]')
email_box.send_keys("0886571626")
password_box = driver.find_element(by=By.XPATH, value='//*[@id="password_input_with_placeholder"]/input')
password_box.send_keys("hoahoc123456")
password_box.send_keys(Keys.ENTER)
time.sleep(5)

with open('my_cookie.pkl','wb') as file:
    pickle.dump(driver.get_cookies(), file)

driver.close()

C:\Users\vuong\AppData\Local\Temp\ipykernel_4584\4041407580.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromePath)


In [3]:
# Hàm lấy link của mỗi bài viết
def Get_PostLink(driver, number_page=6):
    # click vào ô Dòng thời gian để xem các bài viết
    Dongthoigian = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div[1]/div[4]/a[1]')
    Dongthoigian.click()
    
    links = []
    for number in range(1,number_page+1):
        if (number==1):
            for i in range(1,6):
                xpath = f'/html/body/div/div/div[2]/div/div[1]/div[3]/div[2]/div/section/article[{i}]/footer/div[2]/a[1]'
                links.append(driver.find_element(by=By.XPATH, value=xpath).get_attribute('href'))
            driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div[3]/div[2]/div/div[1]/a').click()
            time.sleep(5)
        else:
            for i in range(1,11):
                xpath = f'/html/body/div/div/div[1]/div/table/tbody/tr/td/div/section/article[{i}]/footer/div[2]/a[1]'
                links.append(driver.find_element(by=By.XPATH, value=xpath).get_attribute('href'))
            driver.find_element(by=By.XPATH, value='/html/body/div/div/div[1]/div/table/tbody/tr/td/div/div[1]/a').click()
            time.sleep(5)
    return links

In [18]:
# Hàm lấy nội dung và bình luận của một bài viết
def Get_ContentAndComment(url):
    driver.get(url)
    time.sleep(6)
    # Lấy nội dung bài viết
    content_element = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div/div')
    content = content_element.text
    # Lấy bình luận của bài viết
    comment = []
    for i in range(1,11):
        comment_xpath = f'/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div[{i}]/div/div[1]'
        try:
            comment_element = driver.find_element(by=By.XPATH, value=comment_xpath)
            comment.append(comment_element.text)
        except:
            break
    return content, comment

In [19]:
# Đăng nhập facebook bằng cookie vừa lấy được
driver=webdriver.Chrome(chromePath)
driver.get('https://mbasic.facebook.com/UITconfess')

cookies = pickle.load(open("my_cookie.pkl","rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.get('https://mbasic.facebook.com/UITconfess')
time.sleep(3)


# Crawl dữ liệu
contents, comments = [], []
links = Get_PostLink(driver)
time.sleep(10)
for link in links:
    content, comment = Get_ContentAndComment(link)
    contents.append(content)
    comments.append(comment)
    
driver.close()

C:\Users\vuong\AppData\Local\Temp\ipykernel_17820\3519424675.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromePath)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div/div"}
  (Session info: chrome=112.0.5615.86)
Stacktrace:
Backtrace:
	(No symbol) [0x0097E3E3]
	(No symbol) [0x00913B31]
	(No symbol) [0x00824FE8]
	(No symbol) [0x00850399]
	(No symbol) [0x0085059B]
	(No symbol) [0x0087F062]
	(No symbol) [0x0086B104]
	(No symbol) [0x0087D77C]
	(No symbol) [0x0086AEB6]
	(No symbol) [0x00847178]
	(No symbol) [0x008482DD]
	GetHandleVerifier [0x00BF3D32+2505650]
	GetHandleVerifier [0x00C292A5+2724133]
	GetHandleVerifier [0x00C2DE3C+2743484]
	GetHandleVerifier [0x00A25B30+612784]
	(No symbol) [0x0091C852]
	(No symbol) [0x00922128]
	(No symbol) [0x0092220B]
	(No symbol) [0x0092C9EB]
	BaseThreadInitThunk [0x77297D49+25]
	RtlInitializeExceptionChain [0x7795B74B+107]
	RtlClearBits [0x7795B6CF+191]


In [21]:
# Lưu dữ liệu vào file
df = pd.DataFrame(zip(contents, comments), columns=['Contents', 'Comments'])
file_name = "uit_crawl.csv"
df.to_csv(file_name, sep='\t')

In [22]:
df

,Contents,Comments
0,"#66\nXin chào các bạn đang học UIT nha, mình l...","[#66 tip tuyệt đỉnk nà iu mìn 😇, #67 xu hướng ..."
1,#65\nCho em hỏi điểm xét tuyển phương thức ưu ...,"[cho mình hỏi luôn kĩ thuật phần mềm nữa ạ, AT..."
2,#62\nNho anh Nam qua di mat! Anh nhan tin cho ...,"[Thanh Mỹ =)), Ai nhớ kìa Nam Nguyen]"
3,"#62\n""Xin phép ad duyệt bài này giúp em ạ.\nCU...",[Bạn hãy tin và quyết tâm với sự lựa chọn của ...
4,#60\nem có để quên 1 tai nghe giống vậy ở trườ...,"[#61 gặp tớ là tớ chạy ra xin r😂, #61 thích th..."
5,"#59\nHôm 1/4/2023, mình gặp được bạn Huyền Tra...",[Chang Chang Huỳnh Bông Dao Vinh Thinh Minh Th...
6,#58\nAi đó khuyên bạn H*** khoa KHMT sống khiê...,[m bịa chuyện làm j v Le Hoang]
7,"#57\nMọi người ơi, mình là sinh viên trường kh...","[liên hệ mình nha, cam kết ít hơn tiền học lại..."
8,#54\nBạn Nguyễn Vân Anh lớp N13 có người ấy ch...,[#55 bạn lên thử pctsv nheeeee 🙆🏼‍♀️ tui thấyy...
9,"#52\nEm chào mọi người,\nEm đang là sinh viên ...",[Nguyễn Vinh truyền kinh nghiệm bị bắt ko lập ...
